<a href="https://colab.research.google.com/github/DmitryMok/od_helper/blob/main/prepare_for_od.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Модуль для анализа и конвертации датасета изображения

1. Анализ датасета (размеры и ориентация изображений)
2. Анализ аннотаций (наличие аннотаций для всех изображений)
3. Конвертация в формат yolo
4. Фильтрация изображений (по размерам, количеству изображений и т.п.)

# Анализ датасета

In [ ]:
!git clone https://github.com/DmitryMok/wine_helper # вспомогательные функции
from wine_helper.prepare_data_helper import *

Cloning into 'wine_helper'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 45 (delta 14), reused 33 (delta 9), pack-reused 0
Unpacking objects: 100% (45/45), done.


In [ ]:
!pip install xmltodict  # устанавливаем библиотеку для работы с xml

In [ ]:
import xmltodict

# Функциии

**вывод несколько изображений из папки**

In [ ]:
def draw_imgs_from_path(dir, num=8):
  '''
  Draws images from the folder as a grid (4 cols x num / 4 rows)
  :dir: path to folder with images
  :num: number of images (default = 16)
  '''
  # получим список с именами всех картинок, находящихся в папке pic
  pictures = os.listdir(dir)
  # Упорядочим список 
  pictures = sorted(pictures)
  # Создадим фигуру размером 16 на 4 дюйма
  pic_box = plt.figure(figsize=(32,5*(num//4+1)))
  
  # Поочередно считываем в переменную picture имя изображения из списка pictures. В переменную i записываем номер итерации
  for i, picture in enumerate(pictures[:num]):
      # считываем изображение в picture
      picture = cv2.imread(DIR_IMG + picture)
      # конвертируем BGR изображение в RGB
      picture = cv2.cvtColor(picture, cv2.COLOR_BGR2RGB)
      # добавляем ячейку в pix_box для вывода текущего изображения
      pic_box.add_subplot(num//4+1,4,i+1)
      plt.title(pictures[i] + '\n' + str(picture.shape))
      plt.imshow(picture)
      # отключаем отображение осей
      plt.axis('off')
  # выводим все созданные фигуры на экран
  plt.show()    


In [ ]:
def check_img_dir(dir):
  '''
  Counts images in the folder
  shows file extensions and a graph of image sizes
  :dir: path to folder with images
  '''
  wh_lst = []   # массив для размеров изображений
  img_lst = os.listdir(dir)
  flag = True # флаг мониторинга контроля ожидания
  print('Количество файлов в папке:', len(img_lst))
  print('Расширения:', *{img_name.split('.')[-1] for img_name in img_lst})
  new_time = time.time()
  for i,img_name in enumerate(img_lst[:]):
    tmp_img = cv2.imread(DIR_IMG+img_name)
    wh_lst.extend(tmp_img.shape[:2])
    # print(img_name, tmp_img.shape[:2])
    if flag and time.time()-new_time > 1:
      print('\n...идет обработка, подождите еще примерно', round((time.time()-new_time)*len(img_lst)/(i+1)), 'сек\n')
      flag = False

  with plt.style.context('dark_background'):
    plt.title('Размеры изображений')
    plt.plot(wh_lst[1::2], wh_lst[0::2],':o')


### Для YOLOv5

In [ ]:
# создание папок для yolo

def create_dirs_for_yolo(root_dir):
  '''
  Creates root_dir/train/images, root_dir/train/labels and the same for /val
  '''
  os.makedirs(DIR_TRAIN_YOLO+'train/images/', exist_ok=True)
  os.makedirs(DIR_TRAIN_YOLO+'train/labels/', exist_ok=True)

  os.makedirs(DIR_TRAIN_YOLO+'val/images/', exist_ok=True)
  os.makedirs(DIR_TRAIN_YOLO+'val/labels/', exist_ok=True)

In [ ]:
# создаем custom.yaml
def create_yaml(fname='custom.yaml', classes={0:'person'}, main_dir='/content/dataset/', dirs=['train','val']):
  '''
  Creates custom.yaml for yolov5
  :fname: yaml file name 
  :classes: dictionary of all classes (default {0:'person'})
  :main_dir: path to root folder
  :dirs: train and val subfolders
  '''
  file = open(fname, 'a')  # файл будет создан, если отсутствует

  file.write(f"train: {main_dir+dirs[0]}/\n")
  file.write(f"val: {main_dir+dirs[1]}/\n\n")

  file.write(f"nc: {len(classes)}\n\n")

  file.write(f"names: {list(classes.values())}")
  # names: ['person']
  file.close()

In [ ]:
print('\nфункции успешно загружены')
print('используйте функции:')
print('draw_imgs_from_path - для вывода сетки изображений из папки (файлы сортируются по имени)')
print('check_img_dir - чтобы получить количество изображений, типы расширений и разброс размеров на графике')
print('create_dirs_for_yolo - подготовить папки для датасета на котором будет обучаться yolo')
print('create_yaml - для создания файла yaml yolo с описанием путей и классов')

